In [19]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler




import xlrd
import pandas as pd

book = xlrd.open_workbook('kinetic data for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,385)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)

print(scaler.mean_, scaler.var_)

book = xlrd.open_workbook('prepareddatakinetic.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,7)] for r in range(0,19201)]

standardised_data = scaler.transform(replicated_data)
np.random.shuffle(standardised_data)
array1 = standardised_data[:, 0:4]
array2 = standardised_data[:, 4:7]
x = torch.tensor(array1)
y = torch.tensor(array2)

print(x)
print(y)


[ 4.50062519e-01  2.81646514e+02  6.91231398e-01  3.66233766e+02
  1.19164955e-03 -1.48756240e+00  3.41303629e-03] [3.22038270e-02 5.73513373e+04 2.10258006e-01 2.71715298e+04
 2.82422283e-06 8.08435506e-01 1.62027682e-05]
tensor([[ 0.2137, -0.9813,  0.3443, -1.3118],
        [-1.3892,  2.1468, -1.5075, -0.4018],
        [-0.1467,  1.2952, -0.1105, -1.3118],
        ...,
        [-0.5492, -1.0449, -0.4156, -1.3118],
        [ 0.5318, -0.3362,  0.6189, -1.3118],
        [-0.1480,  1.6069, -0.2192,  1.4182]], dtype=torch.float64)
tensor([[-0.6638,  0.2154, -0.4595],
        [ 0.2894,  1.3408,  0.3171],
        [ 0.0759, -0.8830,  0.2525],
        ...,
        [-0.1063,  0.8372, -0.2908],
        [-0.0581, -0.8154, -0.0148],
        [ 1.5723,  1.8359,  1.7354]], dtype=torch.float64)


In [20]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [21]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 50, 4, 12, 9, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss(reduction='sum')
train = torch.utils.data.TensorDataset(x, y)
train_loader = torch.utils.data.DataLoader(train, batch_size=N,
                                                           shuffle=True)

In [22]:
model.train()
train_epochs=50
learning_rate=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(train_epochs):
    avg_loss=0
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch.float())
        loss = loss_fn(y_pred, y_batch.float())
        avg_loss += loss/ len(train_loader.dataset)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch,avg_loss)

0 tensor(3.9142, grad_fn=<AddBackward0>)
1 tensor(3.1875, grad_fn=<AddBackward0>)
2 tensor(2.1254, grad_fn=<AddBackward0>)
3 tensor(1.7027, grad_fn=<AddBackward0>)
4 tensor(1.4960, grad_fn=<AddBackward0>)
5 tensor(1.3618, grad_fn=<AddBackward0>)
6 tensor(1.2706, grad_fn=<AddBackward0>)
7 tensor(1.2051, grad_fn=<AddBackward0>)
8 tensor(1.1543, grad_fn=<AddBackward0>)
9 tensor(1.1091, grad_fn=<AddBackward0>)
10 tensor(1.0633, grad_fn=<AddBackward0>)
11 tensor(1.0157, grad_fn=<AddBackward0>)
12 tensor(0.9690, grad_fn=<AddBackward0>)
13 tensor(0.9326, grad_fn=<AddBackward0>)
14 tensor(0.9076, grad_fn=<AddBackward0>)
15 tensor(0.8901, grad_fn=<AddBackward0>)
16 tensor(0.8778, grad_fn=<AddBackward0>)
17 tensor(0.8683, grad_fn=<AddBackward0>)
18 tensor(0.8612, grad_fn=<AddBackward0>)
19 tensor(0.8557, grad_fn=<AddBackward0>)
20 tensor(0.8513, grad_fn=<AddBackward0>)
21 tensor(0.8472, grad_fn=<AddBackward0>)
22 tensor(0.8451, grad_fn=<AddBackward0>)
23 tensor(0.8412, grad_fn=<AddBackward0>)
24

In [23]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANNkinetic.pt"))                
                                

[[0.01783333333333333, 24.988333333333344, 0.050833333333333335], [0.023416666666666665, -32.44333333333335, 0.04166666666666668], [0.02313333333333334, -9.365833333333322, 0.041666666666666664], [0.020691666666666674, 32.817499999999995, 0.034999999999999976], [0.019733333333333325, -26.63583333333334, 0.03500000000000003], [0.015975, 132.69500000000005, 0.02749999999999997], [0.01776666666666667, 242.3891666666666, 0.027500000000000007], [0.012699999999999989, 146.2941666666667, 0.029166666666666674], [0.013891666666666672, 193.01666666666657, 0.01416666666666666], [0.01153333333333332, 129.1691666666667, 0.025000000000000022], [0.009850000000000025, 118.19666666666672, 0.01833333333333335], [0.010924999999999999, 140.48833333333323, 0.00916666666666662]]
